# 7.2 케라스 콜백과 텐서보드를 사용한 딥러닝 모델 검사와 모니터링

## 7.2.1 콜백을 사용하여 모델의 훈련 과정 제어하기

### keras의 callback
- callback은 모델의 `fit()`이 호출될 때 전달되는 객체를 뜻함
- 모델의 상태와 성능에 대한 모든 정보에 접근하고 훈련 중지, 모델 저장, 가중치 적재 또는 모델 상태 변경 등을 처리할 수 있음


- callback을 사용하는 몇가지 사례
    - 모델 체크포인트 저장 : 훈련 중에 모델의 가중치를 저장
    - 조기 종료(early stopping) : 특정 모니터링 지표가 더이상 향상되지 않을 때 훈련을 중지(가장 좋은 결과를 얻은 모델을 저장)
    - 훈련 중 하이퍼파라미터 값을 동적으로 조정 : 훈련 중 optimizer의 learning rate를 조정
    - 훈련과 검증 지표를 log에 기록하거나 모델의 가중치가 업데이트 될 때마다 시각화 : 케라스의 진행 표시줄(progress bar)이 하나의 콜백
  
    
- 여러 내장 콜백들은 `keras.callbacks` 모듈에 있음


- 참고) 자동으로 추가되는 콜백들
    - `fit()`메서드가 반환하는 history 객체를 위한 `History` callback
    - 측정 지표의 평균을 계산하는 `BaseLogger` callback
    - `fit()` 메서드에 `verbose=0`을 지정하지 않았다면 진행 표시줄을 위한 `ProgbarLogger` callback
    

### ModelCheckpoint와 EarlyStopping 콜백

- `EarlyStopping` callback을 사용하면 epoch 동안 모니터링 지표가 향상되지 않을 때 훈련을 중지할 수 있음
- 일반적으로 훈련 중 모델을 저장하는 `ModelCheckpoint` callback과 같이 사용됨

In [ ]:
# ModelCheckpoint와 EarlyStopping의 사용 예

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import RMSprop

# 여러 callback들을 리스트로 전달할 수 있음
callbacks_list = [
    # 검증 정확도(val_acc)가 10 epoch보다 더 긴시간동안 향상되지 않으면 훈련을 중지
    EarlyStopping(
        monitor='val_acc', # 기본값은 monitor='val_loss'
        patience=10
    ),
    
    # 훈련 중 검증 손실(val_loss)이 가장 좋을 때마다 계속해서 저장
    # (최종적으로 가장 좋은 모델만 저장됨)
    ModelCheckpoint(
        filepath='my_model.h5',
        monitor='val_loss',
        save_best_only=True
    )
]

model.compile(optimizer=RMSprop(),
              loss='binary_crossentropy',
              # 정확도를 모니터링 하므로 반드시 모델 지표에 포함되어야 함
              metrics=['acc'])


model.fit(x_train, y_train,
          epochs=100,
          batch_size=32,
          callbacks=callbacks_list,
          # 검증 정확도와 검증 손실을 모니터링하므로 검증 데이터가 반드시 전달되어야 함
          validation_data=(x_val, y_val))

### ReduceLROnPlateau 콜백

- ReduceLROnPlateau 콜백을 사용하면 검증 손실이 향상되지 않을 때 학습률을 조절할 수 있음
    - 훈련 중 local minimum에서 벗어나는데 도움이 됨

In [ ]:
# ReduceLROnPlateau 콜백 사용 예

from tensorflow.keras.callbacks import ReduceLROnPlateau

callbacks_list = [
    ReduceLROnPlateau(
        # 검증 손실이 10 epoch동안 좋아지지 않으면 학습률에 0.1을 곱함(학습률을 1/10으로)
        monitor='val_loss',
        factor=0.1,
        patience=10
    )
]

model.fit(x_train, y_train,
          epochs=100,
          batch_size=32,
          callbacks=callbacks_list,
          # 검증 손실을 모니터링하므로 검증 데이터가 반드시 전달되어야 함
          validation_data=(x_val, y_val))

### 자신만의 콜백 만들기

- 내장 콜백에서 제공하지 않는 다른 기능이 필요하면 직접 정의해서 사용할 수 있음


- `keras.callbacks.Callback` 클래스를 상속받아 아래의 약속된 메서드를 구현
    - `on_epoch_begin` : epoch 시작 전 호출
    - `on_epoch_end`   : epoch 끝난 후 호출
    
    - `on_batch_begin` : batch 처리 시작 전 호출
    - `on_batch_end`   : batch 처리 끝난 후 호출
    
    - `on_train_begin` : 훈련 시작 시 호출
    - `on_train_end`   : 훈련 끝날 때 호출
    

- 위 메서드들은 모두 `logs` 매개변수와 함께 호출됨
    - `logs` 매개변수에는 이전 batch, epoch에 대한 훈련 및 검증 측정값이 담겨있는 딕셔너리가 전달됨


- 콜백은 아래의 속성을 참조할 수 있음
    - `self.model` : 콜백을 호출하는 모델 객체
    - `self.validation_data` : `fit()` 메서드에 전달된 검증 데이터

In [ ]:
# 사용자 정의 콜백 예

# 매 epoch의 끝에서 검증 세트의 첫번째 샘플을 입력으로
# 모든 층의 활성화 출력을 계산 후 넘파이 배열로 저장하는 콜백의 예

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback

import numpy as np

class ActivationLogger(Callback):
    
    # set_model()은 호출하는 모델의 정보를 전달하기 위해 훈련 전에 호출됨
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        # 
        self.activations_model = Model(model.input, layer_outputs)
        
    
    def on_epoch_end(self, epoch, logs=None):
        if self.activation_data is None:
            raise RuntimeError('Requires validation data')
            
        # 검증 데이터의 첫번째 샘플을 가져옴
        # [0:1]은 데이터, 레이블 둘 모두 가져온다는 의미
        # 첫번째 원소 : 입력 데이터, 두번째 원소 : 레이블
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
#         f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb')
#         np.savez(f, activations)
#         f.close()
        with open('activations_at_epoch_' + str(epoch) + '.npz', 'wb') as f:
            np.savez(f, activations)